In [2]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.sparse
pd.set_option("display.max_columns",101)
RANDOM_STATE = 42

In [3]:
DATA_PATH = "../data/instacart/"

In [4]:
print('loading prior')
priors = pd.read_csv(DATA_PATH + 'order_products__prior.csv', dtype={
            'order_id': np.int32,
            'product_id': np.uint16,
            'add_to_cart_order': np.int16,
            'reordered': np.int8})

print('loading train')
train = pd.read_csv(DATA_PATH + 'order_products__train.csv', dtype={
            'order_id': np.int32,
            'product_id': np.uint16,
            'add_to_cart_order': np.int16,
            'reordered': np.int8})

print('loading orders')
orders = pd.read_csv(DATA_PATH + 'orders.csv', dtype={
        'order_id': np.int32,
        'user_id': np.int32,
        'eval_set': 'category',
        'order_number': np.int16,
        'order_dow': np.int8,
        'order_hour_of_day': np.int8,
        'days_since_prior_order': np.float32})

print('loading products')
products = pd.read_csv(DATA_PATH + 'products.csv', dtype={
        'product_id': np.uint16,
        'order_id': np.int32,
        'aisle_id': np.uint8,
        'department_id': np.uint8},
        usecols=['product_id', 'aisle_id', 'department_id'])

loading prior
loading train
loading orders
loading products


In [5]:
print('priors {}: {}'.format(priors.shape, ', '.join(priors.columns)))
print('orders {}: {}'.format(orders.shape, ', '.join(orders.columns)))
print('train {}: {}'.format(train.shape, ', '.join(train.columns)))
print('products {}: {}'.format(products.shape, ', '.join(products.columns)))

priors (32434489, 4): order_id, product_id, add_to_cart_order, reordered
orders (3421083, 7): order_id, user_id, eval_set, order_number, order_dow, order_hour_of_day, days_since_prior_order
train (1384617, 4): order_id, product_id, add_to_cart_order, reordered
products (49688, 3): product_id, aisle_id, department_id


In [6]:
print("computing product f")
prods = pd.DataFrame()
prods['orders'] = priors.groupby(priors.product_id).size().astype(np.int32)

computing product f


In [7]:
prods['reordered'] = priors['reordered'].groupby(priors.product_id).sum().astype(np.float32)
prods['reorder_rate'] = (prods.reordered / prods.orders).astype(np.float32)
products = products.join(prods, on='product_id')
products.set_index('product_id', drop=False, inplace=True)
del prods

In [8]:
print ('add order info to priors')
orders.set_index('order_id', inplace=True, drop=False)
priors = priors.join(orders, on='order_id', rsuffix='_')
priors.drop('order_id_', inplace=True, axis=1)

add order info to priors


In [9]:
priors.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,33120,1,1,202279,prior,3,5,9,8.0
1,2,28985,2,1,202279,prior,3,5,9,8.0
2,2,9327,3,0,202279,prior,3,5,9,8.0
3,2,45918,4,1,202279,prior,3,5,9,8.0
4,2,30035,5,0,202279,prior,3,5,9,8.0


In [10]:
print('computing user f')
usr = pd.DataFrame()
usr['average_days_between_orders'] = orders.groupby('user_id')['days_since_prior_order'].\
mean().astype(np.float32)
usr['nb_orders'] = orders.groupby('user_id').size().astype(np.int16)

computing user f


In [11]:
users = pd.DataFrame()
users['total_items'] = priors.groupby('user_id').size().astype(np.int16)
users['all_products'] = priors.groupby('user_id')['product_id'].apply(set)
users['total_distinct_items'] = (users.all_products.map(len)).astype(np.int16)

In [12]:
users = users.join(usr)
del usr

In [13]:
users['average_basket'] = (users.total_items / users.nb_orders).astype(np.float32)
print('user f', users.shape)

('user f', (206209, 6))


In [14]:
print("compute userXproduct f")

compute userXproduct f


In [15]:
priors['user_product'] = priors.product_id + priors.user_id * 100000

In [18]:
priors.shape

(32434489, 11)

In [19]:
i = 0
d = dict()
for row in priors.itertuples():
    i += 1
    if i % 1000000 == 0:
        print i
    z = row.user_product
    if z not in d:
        d[z] = (1, (row.order_number, row.order_id), row.add_to_cart_order * row.order_number)
    else:
        d[z] = (d[z][0] + 1,
               max(d[z][1], (row.order_number, row.order_id)),
               d[z][2] + row.add_to_cart_order * row.order_number)

userXproduct = pd.DataFrame.from_dict(d, orient='index')

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
1390

/Users/coolman/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:13: RuntimeWarning: overflow encountered in short_scalars


9690000
9700000
9710000
9720000
9730000
9740000
9750000
9760000
9770000
9780000
9790000
9800000
9810000
9820000
9830000
9840000
9850000
9860000
9870000
9880000
9890000
9900000
9910000
9920000
9930000
9940000
9950000
9960000
9970000
9980000
9990000
10000000
10010000
10020000
10030000
10040000
10050000
10060000
10070000
10080000
10090000
10100000
10110000
10120000
10130000
10140000
10150000
10160000
10170000
10180000
10190000
10200000
10210000
10220000
10230000
10240000
10250000
10260000
10270000
10280000
10290000
10300000
10310000
10320000
10330000
10340000
10350000
10360000
10370000
10380000
10390000
10400000
10410000
10420000
10430000
10440000
10450000
10460000
10470000
10480000
10490000
10500000
10510000
10520000
10530000
10540000
10550000
10560000
10570000
10580000
10590000
10600000
10610000
10620000
10630000
10640000
10650000
10660000
10670000
10680000
10690000
10700000
10710000
10720000
10730000
10740000
10750000
10760000
10770000
10780000
10790000
10800000
10810000
10820000
10830

In [20]:
userXproduct.head()

,0,1,2
14126415872,5,"(20, 843810)",981
20535312385,1,"(2, 2699553)",18
1375731717,7,"(32, 2844957)",840
8959033352,4,"(42, 1924807)",2194
7342828204,3,"(6, 723315)",63


In [21]:
userXproduct.columns = ['nb_orders', 'last_order_id', 'sum_pos_in_cart']

In [22]:
userXproduct.nb_orders = userXproduct.nb_orders.astype(np.int16)

In [23]:
userXproduct.max_order_number = userXproduct.last_order_id.map(lambda x: x[0]).astype(np.int32)

In [24]:
userXproduct.last_order_id = userXproduct.last_order_id.map(lambda x: x[1]).astype(np.int32)

In [25]:
userXproduct.sum_pos_in_cart = userXproduct.sum_pos_in_cart.astype(np.int16) / userXproduct.max_order_number.astype(np.float32)

In [26]:
print ('user X product f', len(userXproduct))

('user X product f', 13307953)


In [27]:
userXproduct.head()

,nb_orders,last_order_id,sum_pos_in_cart
14126415872,5,843810,49.049999
20535312385,1,2699553,9.000000
1375731717,7,2844957,26.250000
8959033352,4,1924807,52.238094
7342828204,3,723315,10.500000


In [28]:
del priors

In [29]:
users.to_csv(DATA_PATH + "users_match.csv")

In [30]:
products.to_csv(DATA_PATH + "products_match.csv")

In [31]:
userXproduct.to_csv(DATA_PATH + "user_product_match.csv")